# import data

In [1]:
import pandas as pd
import numpy as np

aseg_data = pd.read_csv('../dataset/final_data/aseg+DKT.stats980.csv')
aseg_data.head()

,Group,Age,Sex,Volume_mm30,normMean0,normStdDev0,normMin0,normMax0,Volume_mm31,normMean1,...,Volume_mm398,normMean98,normStdDev98,normMin98,normMax98,Volume_mm399,normMean99,normStdDev99,normMin99,normMax99
0,AD,78,M,239618.359,104.5263,8.7939,23.0,133.0,19019.868,21.8726,...,617.654,78.3675,12.5697,49.0,101.0,4935.454,70.7624,9.5176,42.0,95.0
1,AD,66,M,244462.620,104.4633,9.0946,19.0,135.0,52376.076,18.9785,...,866.656,81.6065,11.9081,49.0,106.0,5642.185,71.9476,9.2368,45.0,97.0
2,AD,77,M,236413.264,104.6093,8.0282,28.0,133.0,16591.926,27.7462,...,928.867,84.8252,10.9218,55.0,101.0,5959.229,76.6012,8.9234,50.0,99.0
3,AD,73,M,227601.449,104.3662,8.7423,29.0,132.0,19109.596,20.5128,...,576.647,80.3540,12.4069,51.0,105.0,5598.818,71.2126,8.7279,43.0,97.0
4,AD,62,M,220511.415,104.5355,7.3271,40.0,129.0,4690.350,36.5242,...,724.538,86.7110,10.1832,61.0,103.0,5327.887,76.5429,7.5031,51.0,96.0


In [2]:
aseg_X = aseg_data.iloc[:, 3:]
aseg_y = aseg_data.iloc[:, 0]
aseg_X = (aseg_X - aseg_X.mean()) / aseg_X.std()
aseg_X = aseg_X.fillna(0)
X, y = aseg_X.to_numpy(), aseg_y.to_numpy()

In [3]:
from sklearn.preprocessing import OneHotEncoder

target2idx = {'AD': 0, 'MCI': 1, 'CN': 2}
idx2target = {0: 'AD', 1: 'MCI', 2: 'CN'}
y_idx = np.array([target2idx[t] for t in y])
y_onehot = OneHotEncoder().fit_transform(y_idx.reshape(-1, 1)).toarray().astype(int)

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

(
    X_train,
    X_test,
    y_train,
    y_test,
    y_train_onehot,
    y_test_onehot,
    y_train_idx,
    y_test_idx,
) = train_test_split(X, y, y_onehot, y_idx, test_size=0.2, random_state=42)

In [5]:
y_train.shape,  y_test.shape

((784,), (196,))

In [6]:
X_test_gnn = X_test.reshape(X_test.shape[0], -1, 5)
X_train_gnn = X_train.reshape(X_train.shape[0], -1, 5)


# spektral

## edge

In [7]:
def get_adjacency_matrix(X):
    # Compute the correlation matrix
    for i in range(X.shape[0]):
        if i == 0:
            corr_matrix = np.corrcoef(X[i])
        else:
            corr_matrix = np.dstack((corr_matrix, np.corrcoef(X[i])))
    # corr_matrix shape: [num_samples, num_nodes, num_nodes]
    return corr_matrix

# adjacency_matrix
adjacency_matrix = (get_adjacency_matrix(X_train_gnn).mean(axis=2) > 0.5).astype(int)
adjacency_matrix[adjacency_matrix == 0] = 1e-10
A_train_gnn = np.tile(adjacency_matrix, (X_train_gnn.shape[0], 1, 1))
A_test_gnn = np.tile(adjacency_matrix, (X_test_gnn.shape[0], 1, 1))

/home/zqy/venv/lib/python3.10/site-packages/numpy/lib/function_base.py:2853: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/zqy/venv/lib/python3.10/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


## Layer

In [8]:
from tensorflow.keras import layers

class AddNorm(layers.Layer):
    def __init__(self, type):
        super(AddNorm, self).__init__()
        if type == 'layer':
            # gat use layer
            self.norm = layers.LayerNormalization()
        elif type == 'batch':
            # gcn use batch
            self.norm = layers.BatchNormalization()
    def call(self, inputs, sublayer):
        # Apply residual connection followed by layer normalization
        return self.norm(inputs + sublayer)

2024-11-10 16:45:26.950963: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-10 16:45:27.059964: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-10 16:45:27.105243: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-10 16:45:27.118213: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-10 16:45:27.198489: I tensorflow/core/platform/cpu_feature_guar

In [9]:
import tensorflow as tf
class PredictionLayer(layers.Layer):
    def __init__(self, num_classes):
        super(PredictionLayer, self).__init__()
        self.ffn = layers.Dense(1, activation='relu')
        self.dense = layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        # x shape: [batch_size, num_nodes, hidden_units]
        x = self.ffn(inputs)
        x = tf.reduce_mean(x, axis=2)
        x = self.dense(x)
        return x

## GCN

In [44]:
from spektral.layers import gcn_conv
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy

import tensorflow as tf
from spektral.layers.convolutional import gcn_conv


class GCN(tf.keras.Model):
    """
    This model, with its default hyperparameters, implements the architecture
    from the paper:

    > [Semi-Supervised Classification with Graph Convolutional Networks](https://arxiv.org/abs/1609.02907)<br>
    > Thomas N. Kipf and Max Welling

    **Mode**: single, disjoint, mixed, batch.

    **Input**

    - Node features of shape `([batch], n_nodes, n_node_features)`
    - Weighted adjacency matrix of shape `([batch], n_nodes, n_nodes)`

    **Output**

    - Softmax predictions with shape `([batch], n_nodes, n_labels)`.

    **Arguments**

    - `n_labels`: number of channels in output;
    - `channels`: number of channels in first GCNConv layer;
    - `activation`: activation of the first GCNConv layer;
    - `output_activation`: activation of the second GCNConv layer;
    - `use_bias`: whether to add a learnable bias to the two GCNConv layers;
    - `dropout_rate`: `rate` used in `Dropout` layers;
    - `l2_reg`: l2 regularization strength;
    - `**kwargs`: passed to `Model.__init__`.
    """

    def __init__(
        self,
        n_labels,
        channels=16,
        activation="relu",
        output_activation="softmax",
        use_bias=False,
        dropout_rate=0.5,
        l2_reg=2.5e-4,
        blocks_num = 3,
        **kwargs
    ):
        super().__init__(**kwargs)
        self.n_labels = n_labels
        self.channels = channels
        self.activation = activation
        self.output_activation = output_activation
        self.use_bias = use_bias
        self.dropout_rate = dropout_rate
        self.l2_reg = l2_reg
        self.blocks_num = blocks_num
        reg = tf.keras.regularizers.l2(l2_reg)
        
        # layer
        for i in range(self.blocks_num):
            setattr(self, f"drop{i}", Dropout(dropout_rate))
            setattr(self, f"gcn{i}", gcn_conv.GCNConv(channels, activation=activation, use_bias=use_bias, kernel_regularizer=reg))
            setattr(self, f"addnorm{i}", AddNorm('batch'))
        self.prediction = PredictionLayer(n_labels)
        # self.drop0 = Dropout(dropout_rate)
        # self.gcn0 = gcn_conv.GCNConv(channels, activation=activation, use_bias=use_bias, kernel_regularizer=reg)

    def get_config(self):
        return dict(
            n_labels=self.n_labels,
            channels=self.channels,
            activation=self.activation,
            output_activation=self.output_activation,
            use_bias=self.use_bias,
            dropout_rate=self.dropout_rate,
            l2_reg=self.l2_reg,
        )

    def call(self, inputs):
        x, a = inputs
        for i in range(self.blocks_num):
            y = getattr(self, f"drop{i}")(x)
            y = getattr(self, f"gcn{i}")([y, a])
            if i == 0:
                x = y
            else:
                x = getattr(self, f"addnorm{i}")(x, y)
                # x = y
        

        # x = self.gcn0([x, a], mask=None)
        # x = self.drop0(x)
        x = self.prediction(x)
        return x

        

In [12]:
from spektral.layers import gcn_conv
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy

import tensorflow as tf
import spektral
from spektral.layers.convolutional import gcn_conv


class GCN2(tf.keras.Model):
    def __init__(
        self,
        n_labels,
        channels=16,
        activation="relu",
        output_activation="softmax",
        use_bias=False,
        dropout_rate=0.5,
        l2_reg=2.5e-4,
        blocks_num = 3,
        ratio = 0.8,
        **kwargs
    ):
        super().__init__(**kwargs)
        self.n_labels = n_labels
        self.channels = channels
        self.activation = activation
        self.output_activation = output_activation
        self.use_bias = use_bias
        self.dropout_rate = dropout_rate
        self.l2_reg = l2_reg
        self.blocks_num = blocks_num
        reg = tf.keras.regularizers.l2(l2_reg)
        self.ratio = ratio
        
        # layer
        for i in range(self.blocks_num):
            setattr(self, f"drop{i}", Dropout(dropout_rate))
            setattr(self, f"gcn{i}", gcn_conv.GCNConv(channels, activation=activation, use_bias=use_bias, kernel_regularizer=reg))
            setattr(self, f"topkpool{i}", spektral.layers.TopKPool(ratio, return_selection=False, return_score=False, sigmoid_gating=False, kernel_initializer='glorot_uniform', kernel_regularizer=None, kernel_constraint=None))
            # setattr(self, f"addnorm{i}", AddNorm('batch'))
        self.prediction = PredictionLayer(n_labels)
        # self.drop0 = Dropout(dropout_rate)
        # self.gcn0 = gcn_conv.GCNConv(channels, activation=activation, use_bias=use_bias, kernel_regularizer=reg)


    def call(self, inputs):
        x, a = inputs
        for i in range(self.blocks_num):
            y = getattr(self, f"drop{i}")(x)
            y = getattr(self, f"gcn{i}")([y, a])
            t = getattr(self, f"topkpool{i}")([y, a])
            if i == 0:
                x = y
            else:
                # x = getattr(self, f"addnorm{i}")(x, y)
                x = y
        

        # x = self.gcn0([x, a], mask=None)
        # x = self.drop0(x)
        x = self.prediction(x)
        return x

        

## train

In [13]:
model = GCN2(n_labels=3, channels=16, blocks_num=1)
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=CategoricalCrossentropy(),
    metrics=[CategoricalAccuracy()],
)
model.build(
    input_shape=[
        (X_train_gnn.shape[1], X_train_gnn.shape[2]),
        (X_train_gnn.shape[1], X_train_gnn.shape[1]),
    ]
)
model.summary()

/home/zqy/venv/lib/python3.10/site-packages/keras/src/layers/layer.py:391: UserWarning: `build()` was called on layer 'gcn2_1', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Model: "gcn2_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dropout_1 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gcn_conv_1 (GCNConv)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ top_k_pool_1 (TopKPool)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ prediction_layer_1              │ ?                      │   0 (unbuilt) │
│ (PredictionLayer)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [14]:
model.fit(
    [X_train_gnn, A_train_gnn],
    y_train_onehot,
    epochs=100,
    batch_size=32,
    validation_data=([X_test_gnn, A_test_gnn], y_test_onehot),
)

Epoch 1/100


/home/zqy/venv/lib/python3.10/site-packages/spektral/layers/convolutional/conv.py:98: UserWarning: The adjacency matrix of dtype <dtype: 'int64'> is incompatible with the dtype of the node features <dtype: 'float32'> and has been automatically cast to <dtype: 'float32'>.
  warnings.warn(


ValueError: Exception encountered when calling TopKPool.call().

[1mDimensions [1,1) of input[shape=[100]] = [] must match dimensions [1,2) of updates[shape=[?,1]] = [1]: Shapes must be equal rank, but are 0 and 1 for '{{node gcn2_1_1/top_k_pool_1_1/TensorScatterUpdate}} = TensorScatterUpdate[T=DT_FLOAT, Tindices=DT_INT32](gcn2_1_1/top_k_pool_1_1/sub_1, gcn2_1_1/top_k_pool_1_1/strided_slice_5, gcn2_1_1/top_k_pool_1_1/strided_slice_1)' with input shapes: [100], [100,1], [?,1].[0m

Arguments received by TopKPool.call():
  • inputs=['tf.Tensor(shape=(None, 100, 16), dtype=float32)', 'tf.Tensor(shape=(None, 100, 100), dtype=int64)']
  • kwargs=<class 'inspect._empty'>

In [34]:
import numpy as np

# Check for null values in X_train_gnn
X_train_gnn_has_null = np.isnan(X_train_gnn).any()
print(f"X_train_gnn has null values: {X_train_gnn_has_null}")

# Check for null values in A_train_gnn
A_train_gnn_has_null = np.isnan(A_train_gnn).any()
print(f"A_train_gnn has null values: {A_train_gnn_has_null}")

X_train_gnn has null values: False
A_train_gnn has null values: False


In [70]:
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
import tensorflow as tf
from spektral.layers import gcn_conv


# Define the GCN model
def create_gcn_model(
    n_labels,
    channels=16,
    activation="relu",
    output_activation="softmax",
    use_bias=False,
    dropout_rate=0.5,
    l2_reg=2.5e-4,
):
    reg = tf.keras.regularizers.l2(l2_reg)

    X_in = Input(shape=(100, 5), name="X_in")
    A_in = Input(shape=(100, 100), name="A_in")

    dropout = Dropout(dropout_rate)(X_in)
    gcn = gcn_conv.GCNConv(
        channels, activation=activation, use_bias=use_bias, kernel_regularizer=reg
    )([dropout, A_in])
    dropout = Dropout(dropout_rate)(gcn)
    output = Dense(n_labels, activation=output_activation)(dropout)

    model = Model(inputs=[X_in, A_in], outputs=output)
    return model


# Create the model
model = create_gcn_model(n_labels=3, channels=16)

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.01),
    loss=CategoricalCrossentropy(),
    metrics=[CategoricalAccuracy()],
)

# Build the model
model.build(
    input_shape=[
        (X_train_gnn.shape[1], X_train_gnn.shape[2]),
        (X_train_gnn.shape[1], X_train_gnn.shape[1]),
    ]
)

# Print the model summary
model.summary()

ValueError: Exception encountered when calling GCNConv.call().

[1mCould not automatically infer the output shape / dtype of 'gcn_conv_18' (of type GCNConv). Either the `GCNConv.call()` method is incorrect, or you need to implement the `GCNConv.compute_output_spec() / compute_output_shape()` method. Error encountered:

Tried to convert 'y' to a tensor and failed. Error: None values not supported.[0m

Arguments received by GCNConv.call():
  • args=(['<KerasTensor shape=(None, 100, 5), dtype=float32, sparse=False, name=keras_tensor_2>', '<KerasTensor shape=(None, 100, 100), dtype=float32, sparse=False, name=A_in>'],)
  • kwargs={'mask': ['None', 'None']}

In [ ]:
# test
import spektral

pool = spektral.layers.TopKPool(0.5)
input_shape = [(10, 16), (10, 10)]
pool.build(input_shape)

In [ ]:
pool.n_nodes, pool._n_nodes